### Ejercicio 2a

Implementamos la función `ping(host)` que envía un único paquete ICMP a un host determinado utilizando la biblioteca Scapy.  
Para cada respuesta recibida, se muestra por terminal:

- La longitud total del paquete recibido
- El tiempo de ida y vuelta (RTT)
- El valor TTL (Time-To-Live) del paquete

Esto permite verificar la recepción correcta de la respuesta ICMP tipo 0 (echo reply), además de obtener información básica de conectividad y latencia con el host de destino.

In [ ]:
from scapy.all import *

#Agrego
from scapy.layers.inet import IP, ICMP
import time
##

def ping(host):
    print (f"Ping {host}")

    #1. Creamos el paquete ICMP
    pkt = IP(dst=host)/ICMP()

    # Medimos el tiempo de envío
    start_time = time.time()

    # Enviamos y recibimos el paquete (timeout de 2 segundos)
    resp = sr1(pkt, timeout=2, verbose=0)
    end_time = time.time()

    if resp is not None and ICMP in resp and resp[ICMP].type == 0:
        # RTT (Round Trip Time)
        rtt = (end_time - start_time) * 1000  # en milisegundos

        # Longitud del paquete
        length = len(resp)

        # TTL (Time to Live)
        ttl = resp.ttl

        print(f"Respuesta de {host}: longitud = {length} bytes | RTT = {rtt:.2f} ms | TTL = {ttl}")
    else:
        print(f"No se recibió respuesta de {host}")

##

ping("google.com")

### PUNTO 2b

Desarrollamos la función `ping_estadisticas(host, cantidad)` que permite enviar múltiples paquetes ICMP a un host utilizando la biblioteca Scapy.  
A partir de las respuestas recibidas, se calculan y muestran las siguientes estadísticas al finalizar el envío:

- Cantidad de paquetes enviados  
- Cantidad de paquetes recibidos  
- Cantidad de paquetes perdidos  
- Porcentaje de paquetes perdidos  
- RTT promedio (Round Trip Time)  
- RTT máximo  
- RTT mínimo  
- Desvío estándar del RTT (utilizando `statistics.stdev()` de Python)



In [ ]:
from scapy.all import *

#Agrego
from scapy.layers.inet import IP, ICMP
import socket ###hay que usar sockets?? lo agrego para hacer bien la resolucion de nombres

import time
import statistics

def ping_estadisticas(host, cantidad):
    print(f"Enviando {cantidad} paquetes ICMP a {host}...\n")

    enviados = 0
    recibidos = 0
    rtts = []

    for i in range(cantidad): ## agrego el fro para asegurarme para asegurarme que el nombre del host tenga ip

        try:
             ip_destino = socket.gethostbyname(host)
        except socket.gaierror:
            print(f"No se pudo resolver el nombre {host}. Abortando...") ##esto es error de DNs
            return

        pkt = IP(dst=ip_destino)/ICMP(id=0x1234, seq=i)

       ## pkt = IP(dst=host)/ICMP(id=0x1234, seq=i)
        enviados += 1

        start_time = time.time()
        resp = sr1(pkt, timeout=2, verbose=0)
        end_time = time.time()

        if resp is not None and ICMP in resp and resp[ICMP].type == 0:
            rtt = (end_time - start_time) * 1000  # RTT en ms
            rtts.append(rtt)
            recibidos += 1
            length = len(resp)
            ttl = resp.ttl
            print(f"[#{i+1}] Respuesta de {resp.src} | Longitud: {length} bytes | RTT: {rtt:.2f} ms | TTL: {ttl}")
        else:
            print(f"[#{i+1}] No se recibió respuesta válida")

    # Estadísticas
    perdidos = enviados - recibidos
    porcentaje_perdidos = (perdidos / enviados) * 100 if enviados > 0 else 0

    print("\nEstadísticas finales:")
    print(f"Paquetes enviados: {enviados}")
    print(f"Paquetes recibidos: {recibidos}")
    print(f"Paquetes perdidos: {perdidos}")
    print(f"Porcentaje de pérdida: {porcentaje_perdidos:.2f}%")

    if rtts:
        print(f"RTT promedio: {statistics.mean(rtts):.2f} ms")
        print(f" RTT máximo: {max(rtts):.2f} ms")
        print(f" RTT mínimo: {min(rtts):.2f} ms")
        if len(rtts) > 1:
            print(f"Desvío estándar del RTT: {statistics.stdev(rtts):.2f} ms")
        else:
            print("Desvío estándar del RTT: No se puede calcular con un solo valor")
    else:
        print("No se puede calcular RTTs: no hubo respuestas válidas")


###INPUT DE PAQUETES
host_input = input("Ingresá el host al que querés hacer ping (ej: google.com): ")
cantidad_input = int(input("¿Cuántos paquetes querés enviar?: "))

ping_estadisticas(host_input, cantidad_input)

## Punto 3

Listamos todos los tipos de errores de ICMP de tipo 3


> Tipo 3 – Destination Unreachable (Destino inalcanzable):

Código 0: Network unreachable

Código 1: Host unreachable

Código 2: Protocol unreachable

Código 3: Port unreachable

Código 4: Fragmentation needed and DF set

Código 5: Source route failed

Código 6: Destination network unknown

Código 7: Destination host unknown

Código 8: Source host isolated

Código 9: Network administratively prohibited

Código 10: Host administratively prohibited

Código 11: Network unreachable for TOS

Código 12: Host unreachable for TOS

Código 13: Communication administratively prohibited

Código 14: Host precedence violation

Código 15: Precedence cutoff in effect


Fuente: RFC 792 -> https://datatracker.ietf.org/doc/html/rfc792


In [ ]:
from scapy.all import *
from scapy.layers.inet import IP, ICMP
import time
import statistics

#errores icmp de tipo 3
ICMP_DEST_UNREACH_CODES = {
    0: "Network unreachable",
    1: "Host unreachable",
    2: "Protocol unreachable",
    3: "Port unreachable",
    4: "Fragmentation needed and DF set",
    5: "Source route failed",
    6: "Destination network unknown",
    7: "Destination host unknown",
    8: "Source host isolated",
    9: "Network administratively prohibited",
    10: "Host administratively prohibited",
    11: "Network unreachable for TOS",
    12: "Host unreachable for TOS",
    13: "Communication administratively prohibited",
    14: "Host precedence violation",
    15: "Precedence cutoff in effect"
}

def ping_estadisticas(host, cantidad):
    print(f"Enviando {cantidad} paquetes ICMP a {host}...\n")

    enviados = 0
    recibidos = 0
    rtts = []

    # Resolución DNS usando Scapy
    try:
        ip_destino = conf.resolve(host)
    except Exception as e:
        print(f"Error de DNS: “Name not known”")
        return

    for i in range(cantidad):
        pkt = IP(dst=ip_destino)/ICMP(id=0x1234, seq=i)
        enviados += 1

        start_time = time.time()
        resp = sr1(pkt, timeout=2, verbose=0)
        end_time = time.time()

        if resp is not None:
            if ICMP in resp:
                icmp_layer = resp[ICMP]
                if icmp_layer.type == 0:
                    rtt = (end_time - start_time) * 1000
                    rtts.append(rtt)
                    recibidos += 1
                    print(f"[#{i+1}] Respuesta de {resp.src} | Longitud: {len(resp)} bytes | RTT: {rtt:.2f} ms | TTL: {resp.ttl}")
                elif icmp_layer.type == 3:
                    code = icmp_layer.code
                    descripcion = ICMP_DEST_UNREACH_CODES.get(code, "Código desconocido")
                    print(f"[#{i+1}] ICMP Error: Destination Unreachable - {descripcion} (Código {code}) desde {resp.src}")
                else:
                    print(f"[#{i+1}] Recibido ICMP tipo {icmp_layer.type}, no es respuesta ni error manejado.")
            else:
                print(f"[#{i+1}] Paquete recibido sin capa ICMP")
        else:
            print(f"[#{i+1}] No se recibió respuesta")

    # Estadísticas
    perdidos = enviados - recibidos
    porcentaje_perdidos = (perdidos / enviados) * 100 if enviados > 0 else 0

    print("\nEstadísticas finales:")
    print(f"Paquetes enviados: {enviados}")
    print(f"Paquetes recibidos: {recibidos}")
    print(f"Paquetes perdidos: {perdidos}")
    print(f"Porcentaje de pérdida: {porcentaje_perdidos:.2f}%")

    if rtts:
        print(f"RTT promedio: {statistics.mean(rtts):.2f} ms")
        print(f"RTT máximo: {max(rtts):.2f} ms")
        print(f"RTT mínimo: {min(rtts):.2f} ms")
        if len(rtts) > 1:
            print(f"Desvío estándar del RTT: {statistics.stdev(rtts):.2f} ms")
        else:
            print("Desvío estándar del RTT: No se puede calcular con un solo valor")
    else:
        print("No se puede calcular RTTs: no hubo respuestas válidas")

# Entradas
host_input = input("Ingresá el host al que querés hacer ping (ej: google.com): ")
cantidad_input = int(input("¿Cuántos paquetes querés enviar?: "))

ping_estadisticas(host_input, cantidad_input)


### Ejercicio 4
Se realizó un test de conectividad utilizando la función `ping_estadisticas()` sobre 5 universidades ubicadas en distintos continentes. El objetivo fue observar la variación en el tiempo de ida y vuelta (RTT) en función de la ubicación geográfica del host.

#### Universidades testeadas (una por continente):

- **América del Sur**: Universidad de Buenos Aires (UBA) — uba.ar
- **América del Norte**: MIT — web.mit.edu
- **Europa**: Universidad de Oxford — ox.ac.uk
- **Asia**: Universidad de Tokio — tsinghua.edu.cn
- **Oceanía**: Universidad de Sídney — unimelb.edu.au


In [1]:
from scapy.all import *
import time

def hacer_ping(host, count=4):
    rtts = []
    for i in range(count):
        pkt = IP(dst=host)/ICMP()
        start_time = time.time()
        resp = sr1(pkt, timeout=2, verbose=0)
        end_time = time.time()

        if resp and ICMP in resp and resp[ICMP].type == 0:
            rtt = (end_time - start_time) * 1000  # ms
            rtts.append(rtt)
    
    if rtts:
        return sum(rtts) / len(rtts)  # promedio
    else:
        return None


# Ejemplo de uso

universidades = {
    "América del Norte": "web.mit.edu",     #USA 
    "América del Sur": "utdt.edu",          #Argentina (UTDT)
    "Europa": "ox.ac.uk",                   #Reino Unido (Oxford)
    "Asia": "en.sjtu.edu.cn",               #Shanghai (SJTU)
    "Oceanía": "unimelb.edu.au"             #Australia (Melbourne)
}

print("RTT promedio a universidades por continente:\n")

for continente, host in universidades.items():
    print(f"{continente} - {host}")
    rtt_prom = hacer_ping(host, count=5)
    if rtt_prom is not None:
        print(f"RTT promedio: {rtt_prom:.2f} ms\n")
    else:
        print("No se recibió respuesta válida\n")



# import subprocess
# import re
# import pandas as pd

# # Lista de universidades por continente
# universidades = {
#     "América del Norte": "web.mit.edu",              # USA
#     "América del Sur": "utdu.edu",                   # Argentina (UTDT)
#     "Europa": "ox.ac.uk",                            # Reino Unido (Oxford)
#     "Asia": "tsinghua.edu.cn",                       # China (Tsinghua)
#     "Oceanía": "unimelb.edu.au",                     # Australia (Melbourne)

# }

# def hacer_ping(host, count=4):
#     try:
#        # output = subprocess.check_output(["ping", "-c", str(count), host], stderr=subprocess.STDOUT, universal_newlines=True)
#         #output = subprocess.check_output([...], stderr=subprocess.STDOUT, universal_newlines=True, encoding="latin1")
#         output = subprocess.check_output([...], stderr=subprocess.STDOUT, universal_newlines=True, encoding="cp1252")

#         match = re.search(r"rtt.* = ([\d\.]+)/([\d\.]+)/([\d\.]+)", output)
#         if match:
#             min_rtt, avg_rtt, max_rtt = map(float, match.groups())
#             return avg_rtt
#         else:
#             return None
#     except subprocess.CalledProcessError as e:
#         print(f"Error con {host}:\n{e.output}")
#         return None

# # Ejecutar ping y guardar resultados
# resultados = []

# for continente, dominio in universidades.items():
#     print(f"Haciendo ping a {dominio} ({continente})...")
#     rtt = hacer_ping(dominio)
#     resultados.append({
#         "Continente": continente,
#         "Universidad": dominio,
#         "RTT promedio (ms)": round(rtt, 2) if rtt else "Error"
#     })

# # Mostrar resultados en tabla
# df = pd.DataFrame(resultados)
# import seaborn as sns
# import matplotlib.pyplot as plt
# import numpy as np

# display(df)

# # Gráfico opcional
# df_valores = df[df["RTT promedio (ms)"] != "Error"]
# df_valores["RTT promedio (ms)"] = df_valores["RTT promedio (ms)"].astype(float)

# sns.barplot(x="Continente", y="RTT promedio (ms)", data=df_valores)
# plt.title("RTT promedio por universidad (ping)")
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()


RTT promedio a universidades por continente:

América del Norte - web.mit.edu
RTT promedio: 26.45 ms

América del Sur - utdt.edu
RTT promedio: 17.51 ms

Europa - ox.ac.uk
RTT promedio: 24.88 ms

Asia - en.sjtu.edu.cn
RTT promedio: 529.56 ms

Oceanía - unimelb.edu.au
RTT promedio: 17.03 ms



### Ejercicio 4a

### Análisis de resultados:

Se observaron diferencias significativas en los RTT medidos. Los tiempos más bajos correspondieron a universidades geográficamente cercanas (UBA y MIT), mientras que los más altos se registraron en universidades ubicadas en Asia y Oceanía.  
Estas diferencias pueden deberse principalmente a:

- La **distancia geográfica** entre el host de origen y el servidor de destino.
- La **cantidad de saltos intermedios (routers)** que deben atravesar los paquetes.
- Las **condiciones de la red** en cada región y la calidad de los enlaces internacionales.
- La **respuesta del propio servidor** (algunas universidades pueden tener filtros o configuraciones que afectan los RTT).


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Resultados obtenidos en WINDOWS
comparacion = [
    {"Continente": "América del Norte", "Universidad": "web.mit.edu", "RTT Scapy":26.45, "RTT Sistema": 12},
    {"Continente": "América del Sur", "Universidad": "utdt.edu", "RTT Scapy": 17.51, "RTT Sistema": 7},
    {"Continente": "Europa", "Universidad": "ox.ac.uk", "RTT Scapy": 24.88, "RTT Sistema": 172},
    {"Continente": "Asia", "Universidad": "hku.hk", "RTT Scapy": 260.4, "RTT Sistema": 255},
    {"Continente": "Oceanía", "Universidad": "unimelb.edu.au", "RTT Scapy": 289.7, "RTT Sistema": 275},
]

df = pd.DataFrame(comparacion)

# Gráfico comparativo
df_melted = df.melt(id_vars=["Continente", "Universidad"], value_vars=["RTT Scapy", "RTT Sistema"],
                    var_name="Fuente", value_name="RTT promedio (ms)")

plt.figure(figsize=(10, 6))
sns.barplot(data=df_melted, x="Continente", y="RTT promedio (ms)", hue="Fuente")
plt.title("Comparación de RTT promedio: Scapy vs Sistema")
plt.ylabel("RTT promedio (ms)")
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Resultados obtenidos en MCBOOK
comparacion = [
    {"Continente": "América del Norte", "Universidad": "web.mit.edu", "RTT Scapy":26.45, "RTT Sistema": 12},
    {"Continente": "América del Sur", "Universidad": "utdt.edu", "RTT Scapy": 17.51, "RTT Sistema": 7},
    {"Continente": "Europa", "Universidad": "ox.ac.uk", "RTT Scapy": 24.88, "RTT Sistema": 172},
    {"Continente": "Asia", "Universidad": "hku.hk", "RTT Scapy": 260.4, "RTT Sistema": 255},
    {"Continente": "Oceanía", "Universidad": "unimelb.edu.au", "RTT Scapy": 289.7, "RTT Sistema": 275},
]

df = pd.DataFrame(comparacion)

# Gráfico comparativo
df_melted = df.melt(id_vars=["Continente", "Universidad"], value_vars=["RTT Scapy", "RTT Sistema"],
                    var_name="Fuente", value_name="RTT promedio (ms)")

plt.figure(figsize=(10, 6))
sns.barplot(data=df_melted, x="Continente", y="RTT promedio (ms)", hue="Fuente")
plt.title("Comparación de RTT promedio: Scapy vs Sistema")
plt.ylabel("RTT promedio (ms)")
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()


A este punto lo automatizo yo desde N8N

Realizar mediciones en diferentes momentos del día (mañana, tarde, noche) hacia la misma universidad durante una semana. ¿Hay diferencias en el RTT para diferentes momentos del día? Si las hay, ¿A qué puede deberse?